In [1]:
from utils import *
import numpy as np
import pickle
import time

In [2]:
t,v = get_data()

In [3]:
ui_mtx = create_ui_matrix(t)

In [31]:
ui_mtx.shape

(6040, 3224)

In [38]:
itzik = ui_mtx[:3,:3]
itzik

array([[4, 5, 5],
       [0, 0, 0],
       [0, 0, 0]])

In [39]:
itzik.mean(axis=1)

array([4.66666667, 0.        , 0.        ])

In [43]:
def func(row):
    return row.sum()*4
np.apply_along_axis(func,1,itzik)

array([56,  0,  0])

In [ ]:
rel_cols = ui_mtx[:,[6,1]]
rel_cols = rel_cols[(rel_cols!=0).all(axis=1)]
rel_cols

In [37]:
c = rel_cols[:,0] - rel_cols[:,1]
c.mean().astype(np.float32)

0.6666667

In [12]:
t[t[:,2] == 0]

array([], shape=(0, 3), dtype=int32)

In [ ]:
test_dic = {'a':1,'b':2,'c':3}
with open('learned_paramaters/dic_test.pickle', 'wb') as f:
    pickle.dump(test_dic, f)

In [ ]:
with open('learned_paramaters/dic_test.pickle', 'rb') as f:
    moti = pickle.load(f)

In [ ]:
moti

In [ ]:
with open('learned_paramaters/popularity_differences_params.pickle', 'rb') as f:
    moti = pickle.load(f)

In [ ]:
len(moti.keys())

In [ ]:
3224*3223

In [2]:
np.random.normal(0, 0.1, (4, 5))

array([[ 0.12577747,  0.13286638, -0.09803337,  0.0981204 ,  0.22344792],
       [-0.03353047,  0.0374203 , -0.00774108, -0.14836026,  0.11435774],
       [ 0.08359585, -0.10718975, -0.00746879, -0.08252795, -0.18735923],
       [-0.06051676, -0.05658918,  0.06794939, -0.14875344, -0.13690487]])

In [19]:
ui_mtx.mean(axis=1).shape

(6040,)

In [4]:
mask = ui_mtx[:,20] * ui_mtx[:,102]
mask[mask>0]

array([16, 12, 16,  4, 12, 16, 15, 16, 12, 12,  9,  6, 12, 12,  4, 12,  6,
        9,  6, 12, 12, 12, 12,  8,  4, 20, 16,  6,  9, 12,  3, 12, 10,  9,
       12,  8,  8])

In [80]:
tsmall = t[:200000, :]
small_ui = create_ui_matrix(tsmall)

In [133]:
temp = small_ui.copy().astype(np.float32)
temp[temp==0] = np.nan
user_means = np.nanmean(temp,axis=1)

In [134]:
import warnings
warnings.filterwarnings("error")

In [135]:
similarity_dict = {}
for i in range(small_ui.shape[0]):
    for j in range(i + 1, small_ui.shape[0]):
        # find shared items between the two users
        shared_items = small_ui[[i,j],:]
        rated_shared_items = shared_items[:,(shared_items != 0).all(axis=0)]
        if rated_shared_items.shape[0] > 0:
            # calculate the similarity between the two users
            a = rated_shared_items[0] - user_means[i]
            b = rated_shared_items[1] - user_means[j]
            mechane = (np.sqrt(np.sum(a ** 2)) * np.sqrt(np.sum(b ** 2)))
            if mechane == 0:
                continue
            else:
                sim = np.sum(a*b) / mechane


            # save similiraty if its bigger than 0
            if sim > 0:
                similarity_dict[(i,j)] = sim.astype(np.float16)

In [136]:
chosen = 7
item = 6
sims = np.zeros(small_ui.shape[0])
for i in range(small_ui.shape[0]):
    if chosen < i:
        sims[i] = similarity_dict.get((chosen,i), 0)
    else:
        sims[i] = similarity_dict.get((i,chosen), 0)
sims[chosen] = -1

In [142]:
itzik = np.argsort(sims)[::-1]

In [ ]:
sorted_sims = np.sort(sims)[::-1]

k = 0
i = 0
mone = 0
mechane = 0
while k < 10 and i < sorted_sims.shape[0]:
    i+=1
    if small_ui[k,item] != 0:
        mone += sorted_sims[k] * small_ui[k,item]
        mechane += sorted_sims[k]
        k += 1

print(mone / mechane)

In [123]:
moshik = ui_mtx[[10,14],:]
moshik

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [126]:
moshik[:,(moshik!=0).all(axis=0)]

array([[4, 2, 4, 3, 4, 4, 4, 3, 4, 4, 1, 3, 3, 3, 5, 5, 3, 5, 3, 2, 3, 2,
        3, 4, 5, 3, 3, 5, 4, 2, 4, 5, 3, 5, 1, 5, 5, 3, 3, 5, 3, 4, 4, 2,
        3, 2, 4, 4, 1, 3, 5, 4, 3, 4, 4, 5, 2, 2, 5, 4, 5, 3, 5, 4, 3, 4,
        3, 4, 5, 4, 3, 3, 4, 5, 2, 2, 4, 2],
       [5, 4, 5, 5, 5, 5, 4, 5, 5, 5, 5, 4, 4, 5, 5, 5, 3, 5, 5, 3, 3, 4,
        5, 4, 5, 5, 4, 5, 4, 2, 5, 5, 4, 5, 1, 5, 4, 3, 1, 5, 4, 5, 5, 4,
        5, 4, 4, 5, 3, 4, 5, 4, 3, 3, 4, 5, 3, 3, 3, 4, 5, 4, 4, 3, 5, 4,
        5, 1, 4, 3, 4, 4, 3, 4, 4, 3, 4, 2]])

In [146]:
t.shape

(859395, 3)